# BrainPy

In this chapter, we will introduce how to realize computational neuroscience model with BrainPy briefly. For more detailed documents and tutorials, please check our github repository [BrainPy](https://github.com/PKU-NIP-Lab/BrainPy) and [BrainModels](https://github.com/PKU-NIP-Lab/BrainModels).

`BrainPy` is a Python platform for computational neuroscience and brain-inspired computation. We can roughly divide the process of modeling with BrainPy into 3 steps:

1) Users define Python classes which inherits from different super classes to demonstrate the neuron models and synapse models they want to use. In class definations, users define object methods with regard of the formats specified by BrainPy, explain what they want the models to do at each time step during the simulation. In this process, BrainPy will assist users in functions like numerical integration, computation of differential equations (ODE, SDE, etc.) and adaptation of various backend (`Numpy`, `PyTorch`, etc.), simplify the code logic.

2) After the definition of neuron models and synapse models, users can instantiate those Python classes as objects of neuron group and synapse connection groups, pass the instantiated objects to the constructor of BrainPy class `Network`, and call `run` method to simulate the network.

3) Call BrainPy modules like `measure` module and `visualize` module to display the simulation results.

With this overall concept of BrainPy, we will introduce more details in realization with code implementations in the following chapter. As the basic element of neural system is neuron, and synapses connect neurons to build networks, we will introduce neuron models, synapse models and network models in order.

## 1. Neuron models

We will introduce neuron models from complex ones to simple ones. To be precise, neuron models can be classified into three types which covers different accuracy: biophysical models, simplify models and firing rate models.

### 1.1 Biophysical models

#### 1.1.1 Hodgkin-Huxley model

<img src="../images/brainpy_tutorials/1-1.png">	(neuron membrane | NeuroDynamics)
<img src="../images/brainpy_tutorials/1-2.png">	(equivalent circuit diagram | NeuroDynamics)

Fig. 1-1 is a diagram of general neuron membrane, including the phospholipid bilayer and ion channels on the membrane. The Na+ ion concentration is higher out of the neuron, and K+ ion concentration is higher in the neuron. Intracellular potential is  lower than extracellular potential.


Hodgkin and Huxley (1952) recorded the generation of action potential on squid giant axons with voltage clamp technique, and proposed the canonical neuron model called Hudgin-Huxley model (HH model). 

This model is based on the equivalent circuit diagram showed in Fig. 1-2, in which the battery E_L refers to the potential difference across membrane, electric current I refers to the external stimulus, capacitance C refers to the hydrophobic phospholipid bilayer with low conductance, resistance R refers to the resistance correspond to leaky current, i.e. the resistance of all non-specific ion channels. 

As Na+ ion channel and K+ ion channel are important in the generation of action potentials, in HH model these two ion channels are modeled as the two resistances R_Na and R_K in parallel on the right side of the circuit diagram, and the two batteries E_Na and E_K refer to the ion potential differences caused by ion concentration difference of Na+ and K+, respectively.


Consider the Kirchhoff’s first law, that is,  for any node in an electrical circuit, the sum of currents flowing into that node is equal to the sum of currents flowing out of that node, Fig. 1-2 can be modeled as differential equations:

$$ C \frac{dV}{dt} = -(\bar{g}_{Na} m^3 h (V - E_{Na}) + \bar{g}_K n^4(V - E_K) + g_{leak}(V - E_{leak})) + I(t) $$

$$\frac{dx}{dt} = \alpha_x(1-x) - \beta_x , x \in \{ Na, K, leak \} $$


Refer to Fig. 1-2, the left hand of equation 1 refers to the current through capacitor C, the first 3 terms on the right hand refers to currents through Na+ ion channel, current through K+ ion channel and leaky current, respectively, and the 4th term is external current.

In equation 1, HH model introduce the concept of gating variables when computing the current go through ion channels. Among the three gating variables m, n and h in the equation, variables m and h control the state of Na+ ion channel, and variable n control the state of K+ ion channel. The dynamics of m, n and h can be expressed in a Markov-like form, in which α_x refers to the activation rate of gating variable x, and β_x refers to the de-activation rate of x. As showed in equation 3-8, the expressions of α_x and  β_x are fitted by experimental data.

$$ \alpha_m(V) = \frac{0.1(V+40)}{1 - exp(\frac{-(V+40)}{10})}$$

$$ \beta_m(V) = 4.0 exp(\frac{-(V+65)}{18})$$

$$ \alpha_h(V) = 0.07 exp(\frac{-(V+65)}{20})$$

$$ \beta_h(V) = \frac{1}{1 + exp(\frac{-(V + 35)}{10})}$$

$$ \alpha_n(V) = \frac{0.01(V+55)}{1 - exp(\frac{-(V+55)}{10})}$$

$$ \beta_n(V) = 0.125 exp(\frac{-(V+65)}{80})$$

<img src="../images/brainpy_tutorials/1-3.png">（V-t plot of HH model simulation）

The V-t plot of HH model simulated with BrainPy is painted in Fig. 1-3.

During the simulation period, we give the HH neuron a constant external stimulus. Receiving the stimulus, the membrane potential integrates slowly for a while, then spikes to a high value, generates an action potential and resets to resting potential. Two action potentials are unlikely to generate continuously, but there is a certain interval between spikes, called refractory period.

These series of membrane potential change are similar to the typical behavior of neurons found in biophysical experiments, and can be mapped to the equations. For example, the existence of refractory  period is because of the different activation and de-activation rate of gating variables.

How can the users model and simulate HH model with BrainPy? Take HH model as an example, we will explain this process below.


**************************

*Note: Readers should pay special attention to discriminate among the terms “model parameters”, “model variables” in dynamic systems and the terms “parameters”, “variables” in programming.*

In BrainPy, users define neuron and synapse models as Python classes. As shown in Fig. 1-4, all the classes users may use to build neuron and synapse models inherit from one base class `Dynamic System`. Among the subclasses of `DynamicSystem` class, `NeuGroup` class corresponds to neuron groups, `TwoEndConn` class corresponds to synapse connections between two neurons, in the future BrainPy will support more prototypes like `ThreeEndConn` class, which corresponds to synapse connections between three neurons. Each specific neuron model or synapse model (ex. LIF neurons, HH neurons, AMPA synapses, etc.) should inherit from one of the subclasses listed above.

In [ ]:
(1-4) (Class inheritance of BrainPy)

In [ ]:
(code - class + derivative)

In BrainPy, HH neuron model can be implemented as a subclass of `NeuGroup` class. Object of HH class represents a group of neuron, and every single neuron in the group will be compute separately. 

In HH class definition, users may set value to variable `target_backend` to specify backend. The value can be set as `general`, which means this model can be ran on any backend supported by BrainPy; it can also be set as a list of backend names:

In [ ]:
(code -  set backend with list)

The code implementation of model rely on the backend. For example, the acceleration mechanisms of `PyTorch` differs from the one of `Numba`, therefore we may realize the same model differently to maximize the accelerate effect of backend. 

Next, we define the differential equation in `derivative` method. Users pass the model variables, time stamp `t` and other model params in order to `derivative` method, BrianPy will classify this three types of parameters by the incoming parameter order. 

In the method, we compute the right hand of HH equations (1) – (8), get dV/dt, dm/dt, dn/dt, dh/dt and return them in order.

In [ ]:
(1-5) (equations vs. code)

However, if you are familiar with numerical integration method, you may have found that we only return the derivatives in `derivative` method (which is reasonable consider the method’s name). To complete the numerical integration, the derivatives dx/dt should be transform into an update process from x(t) to x(t+1). 

BrainPy will automatically complete this step of transformation, the details will be further explained in the definition of class constructor.

The contructor of HH class `__init__` need three types of incoming parameters:

1)	Parameter 	`size`: the neuron number in an object of this class. Ex. If `size=100`, this object of HH neuron group will include 100 HH neurons.

2)	Parameter list `E_Na` ~ `V_th`: a list of model parameters relies on the model. The number and value of model parameters in this list varies according to the model.

3)	Key word arguments list `**kwargs`: Variable dictionary of key word arguments. `**kwargs` are passed to superclass `NeuGroup` at the end of constructor, and their function are to be introduced later with the code of instantiation.

In [ ]:
(code - __init__)

The body of constructor is simple. First we save all the model parameters in instance variables, and define model variables as instance variables, too.

The difference between model parameters and model variables is, model parameters are only sensitive to the model we use, but model variables also consider the number of neurons in this neuron group. As shown in Fig. 1-6, if 100 HH neuron is included in one object, then each model parameter is a floating point number, while each model variable is saved in memory as a floating point vector with length 100.

In [ ]:
(1-6) (model memory)

The we define the numerical method `integral`. In `derivative` method we have got the derivatives, but the numerical integration method is still unclear. In HH model we call `odeint` provided by BrainPy, recompile derivative method and transform the derivative to variable update process. 

BrainPy supports several numerical integration methods including Euler method, Ronge-Kutta method, etc. Users can choose numerical method with parameter `method`.

In [ ]:
(code – set value of `method`)

At the end of constructor, pass parameter `size` and `**kwargs` to super class `NeuGroup`, finish the initialization of HH class.

In [ ]:
(code-update)

Define `update` method of HH class. Simulation is a series of computing operations on discrete time series, so in `update` method we should define the operations on each time step.

In HH model, we call `integral` method to update variables `V`, `m`, `n`, `h`, judge if there are spikes on each neuron and reset the external input of current moment.

To use HH class, we instantiate a neuron group with 100 HH neurons. The param `monitors` is passed into super class `NeuGroup` as `**kwargs`. All the model variables marked by the monitor list will be recorded by BrainPy during the simulation, and users can access the records after simulation in `neu.mon.*`.

Pass neuron group object `neu` to class `Network`, instantiate a network object `net` including this neuron group. Call method `run` of class `Network`, simulate for 200 ms and give all the neurons  external inputs of amplitude 10 during the simulation.

In [ ]:
(code-visualization)

After simulation, call `visualize` module of BrainPy, paint the V-t plot of HH model.

In [ ]:
(1-7)(HH model V-t plot)

### 1.2 Reduced models

In last section we realized the Hodgkin-Huxley model inspired by biophysical experiments. Although HH mode is precise, it will consume ample computing resources and spend running time on details.

To reduce this consumption on time and resources, sometimes researchers can accept the cost of certain accuracy. The reduced models introduced in this section is simple and easy to compute, while they can still represent the main pattern of neuron behavior.

#### 1.2.1 Leaky Integrate-and-Fire model

The most typical reduced model is the Leaky Integrate-and-Fire model (LIF model) presented by Lapicque (1907). 

LIF model can be seen as a combination of integrate process represented by differential equation and spike process represented by conditional judgment:

In [ ]:
(LIF equation & condition)(1)(2)

Fig. 1-8 paint the V-t plot of LIF model. Compare to the HH model, LIF model does not model the burst of membrane potential when generates a spike.

In [ ]:
(1-8)(LIF model V-t plot)

The `derivative` method of LIF model is simpler than of HH model.

In [ ]:
(code)(derivative)

At the mean time, the `update` method is more complex because of the conditional judgement.

In [ ]:
(code-update and annotation on ref and sp)

Note that we write `update` method in vector form here. If the backend is `Numba`, then we can also write the same LIF model with `prange` loop, for `Numba` support parallel acceleration on `prange` loop:

In [ ]:
(code-update with prange)

#### 1.2.2 Quadratic Integrate-and-Fire model
#TODO

#### 1.2.3 Exponential Integrate-and-Fire model
Exponential Integrate-and-Fire model (ExpIF model) (Fourcaud-Trocme et al., 2003) is more expressive than QuaIF model.

(ExpIF eq & condition) (1)

(1-10) (ExpIF model V-t plot)

In Fig. 1-10, with the model parameter V_T, ExpIF model reproduce the rapid growth of membrane potential before a spike, achieve a better simulation result.

In [ ]:
(code-derivative& update)

#### 1.2.4 Adaptative Exponential Integrate-and-Fire model

To reproduce the adaptation behavior of neurons, researchers add a weight variable w to existing integrate-and-fire models like LIF, QuaIF and ExpIF models. Here we introduce a typical one Adaptative Exponential Integrate-and-Fire model (AdExIF model)(Gerstner et al, 2014).

(AdExIF eq & condition) (1)

(1-10) (AdExIF model V-t plot)

The firing rate of AdExIF neuron decreases over time. It simulates the adaptation of neurons, and is controlled by the weight variable w.

In [ ]:
(code-derivative& update)

#### 1.2.5 Resonate-and-Fire model

Other than the integrators we introduced above, there is another neuron type called resonator.

(1-12) (integrator vs. resonator)

The interactions between ion channels generates the subthreshold oscillation od resonator. To model this oscillation, Resonate-and-Fire model includes two model variables x, y to represent the current-like and voltage-like variables in neurons.

(RF eq and cond)

After a short stimulus is given, paint the trajectory of x and y on complex field, we can see the both variables decaying to zero in a nearly circle trajectory. The voltage-like variable y acting like a resonator here.

(1-13)(RF in complex filed; RF y-t plot)

In [ ]:
(code-derivative and update?)

#### 1.2.6 Hindmarsh-Rose model

To simulate the bursting spike pattern in neurons (i.e. continuously firing in a short interval), Hindmarsh and Rose (1984) proposed Hindmarsh-Rose neuron model, import a third model variable as slow variable to control the bursting of neuron.

(HR eq and cond)

(1-14)(HR model with bursting pattern)

In [ ]:
(code derivative and update)

In Fg. 1-15, the three model variables x, y, z changing on time. Variable z changes much slower than x and y, so z is the slow variable.

We should mention that x and y are changing periodically during the simulation. Can BrainPy help us analysis the reason of this periodicity?

(1-15)

Yes. With the module `analysis` of BrainPy, users can do simple dynamic analysis, including 1D/2D bifurcation analysis, fast-slow variable bifurcation analysis and 2D/3D phase plane drawing.

Here we take 2D phase plane drawing as an example. Passing the differential eqution, target variables, fix variables and parameters to `PhasePlane` class in `analysis` module, we can instantiate a PhasePlane analyzer object based on differential equation.

In [ ]:
(code)(analysis)

Call the `plot_nullcline`, `plot_vector_field`, `plot_fixed_point`, `plot_trajectory` method of phase plane analyzer, users can paint nullcline, vector filed, fixed points and trajectory of the dynamic system.

(1-16)(phase plane)

In HR model, the trajectory of x and y approaches a limit cycle, that’s why these two variables change periodically.

#### 1.2.7 Generalized Integrate-and-Fire model

In [ ]:
#TODO

### 1.3 Firing Rate models

Firing Rate models are simpler than reduced models. In these models, each compute unit represents a neuron group, the membrane potential variable V in single neuron models is replaced by firing rate variable a (or r or /niu). Here we introduce a canonical firing rate unit.

#### 1.3.1 Firing Rate Units

#TODO

## 3. Network models

We have introduced how to model neurons and synapses with BrainPy, now we will introduce two main types of network models: 1) spiking neural networks model and compute each neuron or synapse separately; 2) firing rate networks simplify neuron groups in the network as firing rate units and compute each neuron group as one unit.

### 3.1 Spiking Neural Network

#### 3.1.1 E/I balanced network

In 1990s, biologists found in experiments that neuron activities in brain cortex show a temporal irregular spiking pattern. This pattern exists widely in **xx, xx** area etc., but researchers knew few about the mechanism and function of it.

Vreeswijk and Sompolinsky (1996) proposed E/I balanced network to explain this irregular spiking pattern. The feature of this network is the strong, random and sparse synapse connections between neurons. Because of this feature and corresponding parameter settings, each neuron in the network will receive great excitatory and inhibitory input from within the network, but these two types of input will cancel each other, and maintain the total internal input at a relatively small order of magnitude, which is only enough to generate action potentials.

The randomness and noise in E/I balanced network give an internal input which varies with time and space at the order of threshold potential to each neuron in the network. Therefore, the firing of neurons also has randomness,  ensures that E/I balanced network can generate temporal irregular firing pattern spontaneously.

Vreeswijk and Sompolinsky also suggested a possible function of this irregular firing pattern: E/I balanced network can respond to the changes of external stimulus quickly.

(3-1)()

As shown in Fig. 3-1, when there is no external input, the distribution of neurons’ membrane potentials in E/I balanced network follows a relatively uniform random distribution between resting potential and threshold potential.

When we give the network a small constant external stimulus, there are always some neurons whose membrane potentials fall near the threshold potential. Turn the stimulus on and these neurons will soon meet the threshold, therefore spike rapidly. On the network scale, the firing rate of neurons in the network can adjust rapidly once the input changes.

Simulation suggests that the delay of network response to input is the same order of magnitude as synapse delay, and is significantly less than the delay of a single neuron that facing the same stimulus at resting potential generates a spike.

As a result, we say E/I balanced network may provide a fast response mechanism for neural networks. 

(3-2)(Structure of E/I balanced network)

Fig. 3-2 shows the structure of E/I balanced network:

1)	Neurons: LIF neurons are used in the network. The neurons can be divided into excitatory neurons and inhibitory neurons, the number of two types of neurons are N_E: N_I = 4:1.

In [ ]:
(code-neuron group) 

2)	Synapses: Exponential synapses are used in the network. 4 groups of synapse connections are generated between the two groups of neurons, that is, excitatory-excitatory connection (E2E conn), excitatory-inhibitory connection (E2I conn), inhibitory-excitatory connection (I2E conn) and inhibitory-inhibitory connection (I2I conn). To express the excitatory or inhibitory of the synapse connections, we define synapse weight with different signal.

In [ ]:
(code-synapse conn)

3)	Inputs: All neurons in the network receive a constant input current from outside of network.

In [ ]:
(code-net def and run)

See above section 3.1 and 3.2 for definition of LIF neuron class and Exponential synapse class. Visualize the simulation result of E/I balanced network, the network firing rate changes from strong synchronization to irregular fluctuation.

(3-3)(E/I balanced net raster plot)

#### 3.1.2 Decision Making Network

The modeling of computational neuroscience networks can correspond to specific physiological tasks, like the visual motion discrimination task (Roitman and Shadlen, 2002). In this task, rhesus watch a video in which random dots move towards left or right with definite coherence. Rhesus are required to choose the direction that most dots move to and give their answer by saccade. At the meantime, researchers record the activity of their LIF neurons by implanted electrode.

(3-4) (experiment)

Wang (2002) proposed a decision making network to model the activity of rhesus LIF neurons during decision making period. As shown in Fig. 3-5, this network is based on E/I balanced network, with excitatory neuron and inhibitory neuron number in the proportion 4:1, and maintain the balanced state by adjusting parameters.

Among the excitatory neuron group, two selective subgroup are chosen, both with a size of 0.15 * N_E. These two subgroups are marked as A and B in Fig. 3-5. Other excitatory neuron are non-selective.

(3-5)(structure of decision makingnetwork)

In [ ]:
(code-neuron)

As it is in E/I balanced network, synapses can be classified into E2E connection, E2I connection, I2E connection and I2I connection. Excitatory connections are realized with AMPA synapse, inhibitory connections are realized with GABAa synapse. In order to force the network to make decisions between group A and group B, E2E connections are structured. As shown in Sheet 3-1, the strength of synapse connections is higher in the same selective subgroup, and lower between two subgroups or between selective and non-selective subgroup.

(3-1)()

In [ ]:
(code-synapse)

We give two types of external inputs to the decision making network: 1) Background inputs from other brain areas without specific meaning. Represented as high frequency Poisson input mediated by AMPA synapse; 2) Stimulus inputs from outside the brain, which are given only to the two selective subgroup A and B. Represented as lower frequency Poisson input mediated by AMPA synapse. To simulate the proportion difference of the dots moving to left and right in physiological experiments, the stimulus frequencies given to A and B group have a certain difference.

In [ ]:
(code-input)

During the simulation, group A receive a larger stimulus input than group B (i.e. more random dot move to the direction represented by A), later, considerable differentiation are found between the population activities of the two selective subgroups. In this example, the activity of group A is higher than group B, which means, the network choose the right direction receives higher stimulus.

(3-6)(decision making network)

References:
#TODO